In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 98% 1.05G/1.06G [00:12<00:00, 153MB/s]
100% 1.06G/1.06G [00:12<00:00, 89.3MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [46]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function=preprocess_input
        )

test_datagen = ImageDataGenerator(rescale=1./255,
        preprocessing_function=preprocess_input
        )

train_generator = train_datagen.flow_from_directory(
        '/content/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        '/content/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [47]:
from keras.layers import Rescaling,Resizing

In [48]:
from keras.applications.vgg16 import VGG16,preprocess_input

In [49]:
base_model=VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256,256,3))


In [50]:
model=Sequential()
model.add(Resizing(256,256))
model.add(Rescaling(1./255.))
model.add(base_model)
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [51]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [52]:
base_model.trainable=False
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

In [53]:
model.fit(train_ds,epochs=5,validation_data=test_ds)

Epoch 1/5
625/625 [==============================] - 132s 209ms/step - loss: 0.2322 - accuracy: 0.9034 - val_loss: 0.1535 - val_accuracy: 0.9362
Epoch 2/5
625/625 [==============================] - 134s 213ms/step - loss: 0.1214 - accuracy: 0.9515 - val_loss: 0.2114 - val_accuracy: 0.9122
Epoch 3/5
625/625 [==============================] - 146s 233ms/step - loss: 0.0890 - accuracy: 0.9650 - val_loss: 0.2103 - val_accuracy: 0.9204
Epoch 4/5
625/625 [==============================] - 131s 209ms/step - loss: 0.0575 - accuracy: 0.9784 - val_loss: 0.2083 - val_accuracy: 0.9278
Epoch 5/5
625/625 [==============================] - 130s 207ms/step - loss: 0.0426 - accuracy: 0.9850 - val_loss: 0.1933 - val_accuracy: 0.9326


In [56]:
flag=False
for layer in base_model.layers:
  if(layer.name == 'block5_conv1'):
    flag=True
  layer.trainable=flag

In [57]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_6 (Resizing)       (None, 256, 256, 3)       0         
                                                                 
 rescaling_6 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 32)                1048608   
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 15763329 (60.13 MB)
Trainable params: 10

In [59]:
for layer in base_model.layers:
  print(layer.name,layer.trainable)

input_5 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [61]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
model.fit(train_ds,epochs=10,validation_data=test_ds,initial_epoch=5)

Epoch 6/10
625/625 [==============================] - 132s 208ms/step - loss: 0.0380 - accuracy: 0.9855 - val_loss: 0.1997 - val_accuracy: 0.9334
Epoch 7/10
625/625 [==============================] - 130s 207ms/step - loss: 0.0284 - accuracy: 0.9891 - val_loss: 0.3490 - val_accuracy: 0.9132
Epoch 8/10
625/625 [==============================] - 130s 207ms/step - loss: 0.0261 - accuracy: 0.9902 - val_loss: 0.2423 - val_accuracy: 0.9360
Epoch 9/10
625/625 [==============================] - 145s 231ms/step - loss: 0.0155 - accuracy: 0.9951 - val_loss: 0.2810 - val_accuracy: 0.9324
Epoch 10/10
625/625 [==============================] - 130s 208ms/step - loss: 0.0181 - accuracy: 0.9937 - val_loss: 0.2740 - val_accuracy: 0.9328


In [62]:
model.save('VGG.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
